# exp029
[Notion](https://www.notion.so/exp029-4f4f45b75231433ea84a337562c3851b?pvs=4)  
実際の推論パイプラインを再確認し、cv-lbの乖離の原因がバグではないことを確認する.  
Copy from: exp012.ipynb

In [1]:
import os
import random
import sys
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict
from typing import Tuple, Any, Dict

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm

# リポジトリtopに移動
while os.path.basename(os.getcwd()) != 'rsna-2023':
    os.chdir('../')
    if os.getcwd() == '/':
        raise Exception('Could not find project root directory.')
    
from src.data_io import load_dicom_series
from src.segmentation.model import load_models as seg_load_models
from src.segmentation.trainer import inference as seg_inference
from src.classification.dataset import load_df
from src.image_processing import apply_preprocess, crop_organ, kidney_split, resize_volume, apply_postprocess, kidney_specific, resize_3d, resize_1d
from src.classification.model import load_models as cls_load_models
from src.classification.trainer import inference as cls_inference
from src.metrics import score, create_training_solution, normalize_probabilities_to_one

# Configs

In [2]:
class CFG_INF:
    exp_name = 'exp_029'
    # evaluation時：'train', submission時：'test'
    phase = 'train'
    base_dir = 'data/rsna-2023-abdominal-trauma-detection'
    image_dir = f'data/rsna-2023-abdominal-trauma-detection/{phase}_images'
    # dataframeはこのconfigにもたせ、phaseで対応できるようにする.
    if phase == 'train':
        df = pd.read_csv(os.path.join(base_dir, 'train.csv'))
    elif phase == 'test':
        df = pd.read_csv(os.path.join(base_dir, 'sample_submission.csv'))
    df_series_meta = pd.read_csv(os.path.join(base_dir, f'{phase}_series_meta.csv'))
    image_size = (512, 512)
    # sample submissionで極端にスライス数が少ない場合があるため対応.
    min_slices = 10
    # 推論時間制限のため
    max_slices = 500
    max_series = 2
    model_save_dir = "outputs"
    seg_model_mode = 'final'
    lsk_model_mode = 'final'
    be_model_mode = 'final'

class CFG_SEG:
    exp_name = 'exp_004'
    # model config
    backbone = 'efficientnet-b3'
    n_ch = 1
    expand_ch_dim = False
    n_class = 4 # 学習時は腎臓の左右を区別しないので、5->4
    # hyper params
    init_lr = 1e-3
    min_lr = 1e-6
    weight_decay = 1e-4
    image_size = (512, 512)
    batch_size = 32
    amp = True
    n_epoch = 20
    iteration_per_epoch = 200
    pretrain = True
    freeze_epochs = 1
    noaug_epochs = 1
    # fold config
    n_fold = 6
    include_evaluation = False
    train_folds = 1
    # path
    image_dir = "data/dataset001/train_images"
    mask_dir = "data/dataset001/segmentations"
    model_save_dir = "outputs"
    # other config
    seed = 42
    num_workers = 0
    num_gpus = 2
    progress_bar = True
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class CFG_LSK:
    exp_name = 'exp_024'
    # model config
    # timm backbone
    backbone = 'efficientnet-b4'
    n_ch = 1
    expand_ch_dim = True
    # n_class: healthy, low, high
    n_class = 3
    # hyper params
    init_lr = 1e-4
    min_lr = 1e-6
    weight_decay = 1e-4
    image_size = (128, 128, 160)
    batch_size = 64
    amp = True
    eps = 1e-6
    n_epoch = 20
    pretrain = True
    freeze_epochs = 1
    noaug_epochs = 1
    # fold config
    n_fold = 6
    include_evaluation = False
    train_folds = 1
    # path
    image_dir = "data/dataset002"
    model_save_dir = "outputs"
    # other config
    seed = 42
    num_workers = 4
    num_gpus = 2
    progress_bar = True
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    class_weight=torch.tensor([1.0, 2.0, 4.0]).to(device)

class CFG_BE:
    exp_name = 'exp_026'
    # model config
    # timm backbone
    backbone = 'efficientnet_b4'
    # n_ch: z軸方向のスライス数
    n_ch = 5
    expand_ch_dim = False
    # n_class: bowel_injury, extravasation
    # sample weighted: 4class
    n_class = 4
    label_smoothing = None #Optional(float)
    # hyper params
    init_lr = 5e-5
    min_lr = 1e-6
    weight_decay = 1e-4
    image_size = (512, 512)
    batch_size = 64
    amp = True
    eps = 1e-7
    n_epoch = 18
    iteration_per_epoch = 100
    pretrain = True
    freeze_epochs = 1
    noaug_epochs = 2
    # fold config
    n_fold = 6
    include_evaluation = False
    train_folds = 1
    # path
    image_dir = "data/dataset001"
    model_save_dir = "outputs"
    # other config
    seed = 42
    num_workers = 4
    num_gpus = 2
    progress_bar = True
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # b_healty, b_injury, e_healty, e_injury
    class_weight=torch.tensor([1.0, 2.0, 1.0, 6.0]).to(device)

In [3]:
# organs dict (for SEG and LSK models)
organ_index_dict_inv = {
    0: 'liver',
    1: 'spleen',
    2: 'kidney',
    3: 'bowel'
}
organ_index_dict = {v: k for k, v in organ_index_dict_inv.items()}

# labels dict (for BE models)
label_index_dict_inv = {
    0: 'bowel',
    1: 'extravasation'
}

In [4]:
class Inference:
    """推論パイプライン."""
    def __init__(self,CFG_INF: Any, CFG_SEG: Any, CFG_LSK: Any, CFG_BE: Any):
        self.CFG_INF = CFG_INF
        self.CFG_SEG = CFG_SEG
        self.CFG_LSK = CFG_LSK
        self.CFG_BE = CFG_BE

        self.seg_models = seg_load_models(CFG_SEG, mode=self.CFG_INF.seg_model_mode)
        self.lsk_models = cls_load_models(CFG_LSK, mode=self.CFG_INF.lsk_model_mode)
        self.be_models = cls_load_models(CFG_BE, mode=self.CFG_INF.be_model_mode, framework="timm")
    
    def __call__(self, pid: int) -> tuple:
        """inference process.
        1. load images from dicom files.
        2. create segmentation masks.
        3. create liver, spleen, kidney volumes.
        4. inference lsk models.
        5. inference be models.
        Args:
            pid (int): patient id.
        Return example:
            dict: {
            'pid': 0,
            'bowel_healthy': 0.0,
            'bowel_injury': 0.0,
            'extravasation_healthy': 0.0,
            'extravasation_injury': 0.0,
            'kidney_healthy': 0.0,
            'kidney_low': 0.0,
            'kidney_high': 0.0,
            'liver_healthy': 0.0,
            'liver_low': 0.0,
            'liver_high': 0.0,
            'spleen_healthy': 0.0,
            'spleen_low': 0.0,
            'spleen_high': 0.0
            }
        Note:
            - １症例に複数シリーズ存在する場合、各シリーズに対して推論を行い、全予測結果の最大値を採用する.
            - 推論時間的に厳しければ、最初のシリーズのみを採用するなど検討.
        """
        df_study = self.CFG_INF.df_series_meta[self.CFG_INF.df_series_meta['patient_id']==pid].reset_index(drop=True)
        # df_study内のそれぞれのシリーズを取得して、画像枚数に対して降順にソート.
        df_study = self.get_slices_and_sort(df_study)
        preds = defaultdict(list)
        for sid in df_study['series_id'].to_list()[:self.CFG_INF.max_series]:
            data = self.load_data(pid, sid)
            if data is None:
                continue
            lsk_preds = self.lsk_prediction(data)
            be_preds = self.be_prediction(data)
            
            for idx, organ in organ_index_dict_inv.items():
                if idx == 3:
                    continue
                preds[organ].append(lsk_preds[idx])
            for idx, label in label_index_dict_inv.items():
                pred = np.array([be_preds[idx]])
                preds[label].append(pred)

        ret = {'patient_id': pid}
        for k,v in preds.items():
            v = np.array(v)
            ret[k] = np.max(v, axis=0)
        ret = self.convert_submission_format(ret)
        return ret

    def load_data(self, pid: int, sid: int)-> np.ndarray:
        """dicomから画像を読み込む.
        Args:
            pid (int): patient id.
            sid (int): series id.
        Returns:
            np.ndarray: (Z, H, W) normalized CT series.
        Note:
            - preprocessは全モデル共通なので、ここで行う.
            - H, Wはすべてself.CFG_INF.image_sizeにresizeされる.
        """
        series_path = os.path.join(self.CFG_INF.image_dir, str(pid), str(sid))
        # sample submissionでこういう例が存在する.
        if not os.path.exists(series_path): 
            return None
        image_arr, path_list, meta_list = load_dicom_series(series_path, self.CFG_INF.max_slices)
        image_arr = apply_preprocess(image_arr, resize=self.CFG_INF.image_size)
        # sample submission対応
        if len(image_arr) < self.CFG_INF.min_slices:
            image_arr = resize_1d(image_arr, self.CFG_INF.min_slices, axis=0)
        return image_arr
    
    def get_slices_and_sort(self, df_study: pd.DataFrame)-> pd.DataFrame:
        """シリーズのスライス数を取得して、スライス数に対して降順にソートする.
        Args:
            df_study (pd.DataFrame): series meta dataframe.
        Returns:
            pd.DataFrame: sorted series meta dataframe.
        """
        pid = df_study['patient_id'][0]
        df_study['n_slices'] = 0
        for i in range(len(df_study)):
            sid = df_study['series_id'][i]
            series_path = os.path.join(self.CFG_INF.image_dir, str(pid), str(sid))
            if os.path.exists(series_path):
                df_study['n_slices'][i] = len(os.listdir(series_path))
        df_study = df_study.sort_values(by='n_slices', ascending=False)
        return df_study
    
    def lsk_prediction(self, data: np.ndarray)-> np.ndarray:
        """liver, spleen, kidneyの予測値を返す.
        Args:
            data: (Z, H, W).
        Returns:
            np.ndarray: (organs, grades).
        """
        volumes = self.get_lsk_volumes(data) # (organs, z, h, w)
        lsk_iterator = self.pseudo_iterator(self.CFG_LSK, volumes)
        pred = cls_inference(self.CFG_LSK, self.lsk_models, lsk_iterator)
        return pred

    def get_lsk_volumes(self, data: np.ndarray)->Dict[str, np.ndarray]:
        """Segmentationからliver, spleen, kidneyのvolume dataを作成.
        Args:
            data: (Z, H, W).
        Returns:
            np.ndarray: (organs, z, h, w).
        Note:
            - organsはliver, spleen, kidneyの順番.
            - この関数内でCFG.LSK.image_sizeのreshapeまで行う.
            - 腎臓は左右を分離してからくっつけ直すという特殊な処理が必要.
        """
        masks = self.get_segmentation(data)
        masks = apply_postprocess(self.CFG_SEG, masks)
        arr = []
        for idx, organ in organ_index_dict_inv.items():
            if idx == 3:
                continue
            organ_segment = masks[..., idx]
            if organ_segment.sum() == 0:
                arr.append(np.zeros(self.CFG_LSK.image_size))
                continue
            img_cropped, mask_cropped = crop_organ(data, organ_segment)
            if organ == "kidney":
                kidney_r, kidney_l = kidney_split(img_cropped, mask_cropped)
                img_cropped = kidney_specific(self.CFG_LSK, kidney_r, kidney_l)
            else:
                img_cropped = resize_3d(img_cropped, self.CFG_LSK.image_size)
            arr.append(img_cropped)
        arr = np.stack(arr, axis=0)
        return arr
    
    def get_segmentation(self, data: np.ndarray)->np.ndarray:
        """Segmentation modelを使って、各臓器のマスクを作成.
        Args:
            data: (Z, H, W).
        Returns:
            mask: (z, h, w, ch) binarized."""
        seg_iterator = self.pseudo_iterator(self.CFG_SEG, data)
        pred = seg_inference(self.CFG_SEG, self.seg_models, seg_iterator)
        pred = (pred > 0.5).astype(np.uint8)
        return pred
    
    def be_prediction(self, data: np.ndarray)-> np.ndarray:
        """bowel_injury及びextravasation_injuryの予測を行う.
        Args:
            data: (Z, H, W).
        Returns:
            np.ndarray: [bowel_injury_pred, extravasation_injury_pred].
            example: [0.1, 0.9].
        """
        be_iterator = self.pseudo_iterator(self.CFG_BE, data)
        pred = cls_inference(self.CFG_BE, self.be_models, be_iterator)
        pred = self.be_prediction_postprocess(pred)
        return pred
    
    def be_prediction_postprocess(self, pred: np.ndarray)-> np.ndarray:
        """スライスごとの予測をシリーズの予測に変換する.
        Args:
            pred: (len(data),['bowel_injury', 'extravasation_injury']).
        Returns:
            np.ndarray: ['bowel_injury', 'extravasation_injury'].
        Note:
            - 予測値の最大値から外れ値を考慮した2%percentileを採用する.
        """
        if self.CFG_BE.n_class == 1:
            bowel = np.zeros_like(pred[:, 0])
            extravasation = pred[:, 0]
        elif self.CFG_BE.n_class == 2:
            bowel = pred[:, 0]
            extravasation = pred[:, 1]
        elif self.CFG_BE.n_class == 4:
            bowel = normalize_probabilities_to_one(pred[:, :2])
            extravasation = normalize_probabilities_to_one(pred[:, 2:])
            bowel = bowel[:, 1]
            extravasation = extravasation[:, 1]
        p = 98
        bowel = np.percentile(bowel, p)
        extravasation = np.percentile(extravasation, p)
        return np.array([bowel, extravasation])

    def pseudo_iterator(self, CFG: Any, images: np.ndarray)-> tuple:
        """evaluation iterator.
        Args:
            CFG: config.
            images: (batch dim, H, W) or (batch dim, Z, H, W).
        """
        batch = CFG.batch_size
        length = len(images)
        arr = []
        if not CFG.expand_ch_dim:
            images = self.add_dummy_array(CFG, images)
        for i in range(length):
            if CFG.expand_ch_dim:
                img = images[i]
                img = img[np.newaxis, ...]
            else:
                img = images[i:i+CFG.n_ch]
            arr.append(img)
            if i != 0 and (i%batch==0 or i == length-1):
                arr = np.stack(arr, axis=0)
                arr = torch.from_numpy(arr.astype(arr.dtype, copy=False))
                yield arr
                arr = []
    
    def add_dummy_array(self, CFG: Any, images: np.ndarray)-> np.ndarray:
        """chが複数ある場合に、事前に0配列を追加しておく."""
        add_ch = CFG.n_ch//2
        arr = []
        img = np.zeros_like(images[0])
        for i in range(add_ch):
            arr.append(img)
        arr.extend(images)
        for i in range(add_ch):
            arr.append(img)
        arr = np.stack(arr, axis=0)
        return arr

    def convert_submission_format(self, pred: dict)->dict:
        """提出形式に変換する."""
        converted = dict()
        for idx, organ in organ_index_dict_inv.items():
            if idx == 3:
                continue
            for idx, grade in enumerate(['healthy', 'low', 'high']):
                converted[f'{organ}_{grade}'] = pred[organ][idx]
        for idx, label in label_index_dict_inv.items():
            converted[f'{label}_healthy'] = 1 - pred[label][0]
            converted[f'{label}_injury'] = pred[label][0]

        converted['patient_id'] = pred['patient_id']
        return converted

In [5]:
df_solid_organ = load_df(CFG_LSK)
# fold 0のpatient_idを取得
pids = df_solid_organ[df_solid_organ["fold"] == 0]["patient_id"].unique()
df_all = pd.read_csv(os.path.join(CFG_INF.base_dir, 'train.csv'))
train_pids = df_solid_organ[df_solid_organ["fold"] != 0]["patient_id"].unique()
valid_pids = df_solid_organ[df_solid_organ["fold"] == 0]["patient_id"].unique()
df_train = df_all[df_all["patient_id"].isin(train_pids)].reset_index(drop=True)
df_valid = df_all[df_all["patient_id"].isin(valid_pids)].reset_index(drop=True)

In [6]:
inference_instance = Inference(CFG_INF, CFG_SEG, CFG_LSK, CFG_BE)

In [8]:
results = []
for pid in tqdm(df_valid['patient_id'].to_list()):
    result = inference_instance(pid)
    results.append(result)

100%|██████████| 525/525 [3:20:38<00:00, 22.93s/it]  


In [9]:
import pickle
# resultsを保存
dir_ = os.path.join(CFG_INF.model_save_dir, CFG_INF.exp_name)
os.makedirs(dir_, exist_ok=True)
path = os.path.join(dir_, "results.pkl")
with open(path, 'wb') as f:
    pickle.dump(results, f)

# resultsを読み込み
# with open(path, 'rb') as f:
#    results = pickle.load(f)

In [10]:
submission = pd.DataFrame(results)
order = CFG_INF.df.columns.tolist()
if "any_injury" in order:
    order.remove("any_injury")
submission = submission[order]

In [11]:
submission.head()

,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high
0,10007,0.816518,0.183482,0.947678,0.052322,0.951286,0.029996,0.012964,0.905826,0.060036,0.033985,0.941805,0.036837,0.016900
1,10205,0.779929,0.220071,0.946795,0.053205,0.872198,0.078430,0.053869,0.919182,0.054013,0.024664,0.928935,0.046155,0.023084
2,10275,0.956565,0.043435,0.965257,0.034743,0.953950,0.034398,0.011485,0.944327,0.085028,0.038267,0.970852,0.041635,0.021986
3,10430,0.956799,0.043201,0.844909,0.155091,0.864793,0.135347,0.130747,0.858291,0.108673,0.066383,0.894300,0.077345,0.039788
4,10494,0.816477,0.183523,0.916104,0.083896,0.927674,0.054439,0.019471,0.942114,0.037052,0.006877,0.957192,0.028234,0.008788


In [12]:
# add weight
solution_train = create_training_solution(df_valid)

no_scale_score = score(solution_train.copy(),submission.copy(),'patient_id')
print(f'Training score without scaling: {no_scale_score:.4f}')

bowel: 0.1935
extravasation: 0.8347
kidney: 0.5443
liver: 0.6181
spleen: 0.6675
any_injury: 1.1834
mean: 0.6736
Training score without scaling: 0.6736


In [13]:
# Group by different sample weights
scale_by_1 = ['bowel_injury']
scale_by_2 = ['kidney_low','liver_low','spleen_low']
scale_by_4 = ['kidney_high','liver_high','spleen_high']
scale_by_6 = ['extravasation_injury']

# Scale factors based on described metric 
sf_1 = 0.3
sf_2 = 2
sf_4 = 4
sf_6 = 6

# Reset the prediction
y_pred = submission.copy()

# Scale each target 
y_pred[scale_by_1] *=sf_1
y_pred[scale_by_2] *=sf_2
y_pred[scale_by_4] *=sf_4
y_pred[scale_by_6] *=sf_6

weight_scale_score = score(solution_train.copy(),y_pred.copy(),'patient_id')
print(f'Training score with weight scaling: {weight_scale_score:.4f}')

bowel: 0.1478
extravasation: 0.5995
kidney: 0.5112
liver: 0.6342
spleen: 0.6176
any_injury: 0.7761
mean: 0.5477
Training score with weight scaling: 0.5477
